In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "你的langchain_key"

os.environ["OPENAI_API_KEY"] = "你的openapi key"
os.environ["OPENAI_BASE_URL"] = "你的openai 请求地址"

In [2]:
pip install docx2txt pypdf

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 1.文档加载
from langchain.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader

def load_documents(file_path):
    if(file_path.endswith('.txt')):
        loader = TextLoader(file_path, encoding='utf-8')
    elif (file_path.endswith('.pdf')):
         loader = PyPDFLoader(file_path)
    elif (file_path.endswith('.docx')):
         loader = Docx2txtLoader(file_path)
    else:
         raise ValueError("Unsupported file format")
    document =  loader.load()
    return document
     
def load_direcotry(directory_path):
    documents = []
    for filename in os.listdir(directory_path): 
        file_path = os.path.join(directory_path, filename)
        document = load_documents(file_path)
        documents.extend(document)
    return documents
        
all_documents = load_direcotry('docs')       



In [4]:
print(all_documents[0]) 

page_content='易速鲜花网站是一个服务型的网站，提供的是一个平台和在这个平台上的服务。纵观整个互联网，最能盈利的网站就是那些提供服务的网站。在经过了互联网泡沫之后，互联网产生的新的一种盈利的方式就是提供各种各样的服务。

网站提供的是服务，而不是现在互联网上大多数网站提供的是信息，用户可以到我们的网站来进行一些活动，做他们想做的事情，我们提供的是这个平台，就象市场一样，有很多卖菜的摊子，大家可以进去买菜。在社会高速发展的今天，相信大家都很清楚的了解，没有了互联网我们的生活将会是什么样子，工作、生活、学习等等都已经离不开互联网，但是即便是这样我觉的互联网还是没有充分的利用起来。现在的互联网的用处只不过是大家获得自己需要的信息的一种途径，互联网从产生那一天起就是为了信息的共享这个目的，并不是在上面进行自己的活动，例如组织起来打篮球等等。即便是现在已经有众多的社区、交友型的网站但是发展的也够理想，但是这些网站依旧很火爆。而我们的想法是要把互联网变成一个活动或者是日常生活的场所，在上面你也许获得不到信息或者一些资源，但是你能获得的是跟你生活息息相关的东西。所以网站定位的使用人群是哪些人，必须能够上网并且接受这种全新的生活方式跟社会活动方式。这个思想这样说选好消费人群即针对哪种客户提供什么样的服务。

网站有很多类型的或者是不同功能，而我们做的只是一个分类，网站定位很重要。拼房，现在很多有关房屋的网站都会有人发帖进行合租，但是并没有提供完善的平台，用户对这方面的需求不是很集中，网站就充分的解决了这个问题，给大家一个很好的平台很容易的拼起来。网站就可以提供基于这些拼的平台，对他们进行详细的分类，并且提供了方便用户使用的功能，提供了安全的机制，并且提供了更加方便用户拼的各种功能。加之给大家提供了一个信任、时尚、活跃的环境。网站的各种拼的方式，有助于用户更加好的生活，尝试这种新的生活方式给他们带来的快乐、方便、节省。例如网站的拼项目，其实就是项目合作，这就给了他们寻找一起能合作的项目的平台，扩大人脉。所以网站总的来说就使提供了各种概念的服务，这个平台也同样为用户的安全性、易用性提供了很好的解决方式。根据用户的需求，用户同样也可以自己一些新型的需求。

二、盈利来源

网站具有很好的利润来源，并且这种利润来源的方式并不影响用户使用网站，网站的利润的来源将会全部来自跟网站合

In [5]:
# 2.进行文本切块
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, separators=["\n\n"])
chunks = text_splitter.split_documents(all_documents)
print(len(chunks)) 

40


In [8]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 39.1 kB/s eta 0:00:0000:0300:11
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 3.进行向量化和存储
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS #使用faiss向量库

def create_vector_store(chunks,save_path):
    # 初始化 OpenAI 嵌入模型
    embedding = OpenAIEmbeddings(model="text-embedding-ada-002",chunk_size=1000)
    # 创建FAISS向量库
    vector_store = FAISS.from_documents(documents=chunks,embedding=embedding)
    
    # 保存到本地（可选）
    if save_path:
        vector_store.save_local(save_path)
        print(f"Vector store saved to {save_path}")
    return vector_store

vector_store = create_vector_store(chunks,'./docs')

Vector store saved to ./docs


In [ ]:
# 如果已经有embedding文件，直接加载，不要重新处理了，节省token
embedding = OpenAIEmbeddings(model="text-embedding-ada-002",chunk_size=1000)
vector_store_1 = FAISS.load_local(
    folder_path="./docs",       # 存放index.faiss和index.pkl的目录路径
    embeddings=embedding,      # 必须与创建时相同的嵌入模型
    index_name="index",          # 可选：若文件名不是默认的"index"，需指定前缀
    allow_dangerous_deserialization=True  # 显式声明信任
)
print(f"已加载 {len(vector_store_1.docstore._dict)} 个文档块")
print(vector_store_1)

已加载 40 个文档块


In [ ]:
# 4.准备和Retrieval链
from langchain_openai import ChatOpenAI # ChatOpenAI模型
from langchain.retrievers.multi_query import MultiQueryRetriever # MultiQueryRetriever工具
from langchain.chains import RetrievalQA # RetrievalQA链

# 实例化一个大模型工具 - OpenAI的GPT-3.5
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# 实例化一个MultiQueryRetriever
multi_retriver = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm)  # include_original包含原始查询

# 实例化一个RetrievalQA链
qa_chain = RetrievalQA.from_chain_type(llm,retriever=multi_retriver,chain_type_kwargs={"verbose": True})

In [24]:
# 5. 问答展示
def ask_question(query):
    # 使用RetrievalQA链来获取答案
    response = qa_chain.invoke(query)
    
    # 返回得到的答案
    return response

answer_1=ask_question('应聘人员须提供什么资料')
print(answer_1)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
易速鲜花集团 
 5 
 
第一章 人事管理制度 
第一节 员工的招聘与录用 
 
1、人事部统一将用人部门招人计划呈报总经理签字审批后由人事
部负责招聘。 
2、公司员工分为管理公司员工和代商家招聘的员工。 
3、招聘的基本原则：公平、公正、择优录取。 
4、应聘人员须向我公司提供以下资料： 
① 本人如实填写《入职申请表》一份； 
② 近期免冠标准照片四张； 
③ 身份证复印件（验明正件）； 
④ 学历证明（毕业证书、学位证书及验证证明）； 
⑤ 职称证书和资格证书；  
⑥ 与原单位解除劳动关系的证明文件； 
⑦ 公司指定医院体检证明； 
⑧ 人力资源中心要求提供的其他资料。

易速鲜花集团 
 6 
5、招聘采用面试及笔试的方式进行初选。 
6、经初选合格的人员须参加公司举办的“新员工入职培训”学习，
培训结束经考核合格后，即可成为我公司的试用员工。 
7、进入公司一个月内必须签订劳动合同，根据签订《劳动合同》
的年限，试用期一至六个月，试用期享受试用待遇。 
8、特殊工种需持有相应工作证件方可入职。 
 
第二节 培训 
 
1、公司根据工作需要，随时对员工进行必要的基本素质和专业素
质的在职培训，培训考核结果记入个人档案与绩效挂钩。 
2、员工要认真学习，通过学习提高自己的文化素质和业务水平。 
3、员工试用期间必须通过培训完成公司相关管理制度，本岗位工
作职责、工作流程的学习。 
4、由公司外派参加培训的，须遵守与公司签订培训协议，如有违
反，一切按照协议内容执行。 
5、员工工作期间，参加的各项培训考核中不合格者，将给予一次
补考机会，补考不合格者到人力资源部进行脱岗培训学习。 

In [28]:
# 内存缓存（短期高频）
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

# 1. 首先启用内存缓存（只需执行一次）
set_llm_cache(InMemoryCache())  # 缓存检索结果

# 2. 执行查询（自动缓存）
answer_2=ask_question('Rose的花语是什么')
print(answer_2)

# 3. 再次执行相同查询将直接从缓存读取
answer_2_cached = ask_question('Rose的花语是什么')  # 命中缓存
print(answer_2_cached)  # 结果与第一次相同，但未实际调用LLM




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
易速鲜花集团 
 2 
易速鲜花晨会宣言 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
今天 
我开始新的生活 
我要用全身心的爱迎接今天 
我会用我全部的热情 
关爱我的同事 
热爱我们的公司 
服务于我们的客户 
忠于我们的工作 
在努力工作中充实、完善，并超越自我 
每天进步一点点 
今天 
我就付诸于行动！

易速鲜花集团 
 3 
 
易速鲜花简介 
易速鲜花，一个超越一切鲜花运营商的全新概念，土地犹如一片
绿色的画布，展示着生机勃勃的花海。我们在这里嫁接了乐、吃、住、
行、游等元素，打造出一个独特的，富含中国原生态特色的网上鲜花
服务平台。 
易速鲜花，赋予每一朵鲜花更深的情感内涵。我们的文化展示服
务包括酒文化、茶文化、农耕文化等，将呈现出多行业业态相互辉映
及繁荣共存的繁花似锦。 
易速鲜花，她以高品位的服务，超前的经营理念，尖端的营销策
略和丰富的艺术气质，旨在满足大众的需求，创造独一无二的网上购
花体验。 
易速鲜花，她是古老与现代的完美融合，她将带您体验夜郎古国
的历史风貌，感受穿越时空的神奇。购花不再只是交易，更是一次文
化之旅、美食之旅、音乐之旅，一场美妙体验的新境界。 
易速鲜花，她超越了传统网络鲜花运营商单一的经营模式，她将
成为影响并促进中国花卉行业综合经济发展和地位提升的重要力量，
同时也是一张崭新的名片。 
因此，易速鲜花，她不只是售卖鲜花，她记忆的是人类对自身生
存环境的思考。她带给您的不仅是一束鲜花，更是一切未来的记忆带
给您身心合一的体验，让您在每一次购花过程中都感到心动。 
易速鲜花，她是这座城市的明灯，她激活了这里的活力，